In [1]:
import numpy as np
import os
from keras import optimizers

Using TensorFlow backend.


In [2]:
from Models.PoolingAndFire import create_model, create_loss_function
from PreProcess import data_generator, get_num_samples
from GenerateData import generate_data

In [3]:
DATA_DIR = r"./data"

In [4]:
BATCHSIZE = 16
EPSILON = 1e-16

WEIGHT_DECAY = 0 # 0.001
KEEP_PROB = 0.5
CLASSES = 1

LABEL_WEIGHT = 1.0
OFFSET_LOSS_WEIGHT = 1.0
OFFSET_WEIGHT = 40.0

HEIGHT = 320
WIDTH = 320
CHANNELS = 3

VALIDATION_SPLIT = 0.3

In [5]:
generate_data(DATA_DIR, WIDTH, HEIGHT, box_min=50, box_max=100, num_images=1000)

Generating data in: ./data


In [6]:
model = create_model(320, 320, 3)
out_shape = model.output_shape
anchor_width = out_shape[1]
anchor_height = out_shape[2]
print(f"Needed anchor shape: {anchor_width}x{anchor_height}")

Needed anchor shape: 20x20


In [7]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 320, 320, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 160, 160, 128 3456        input[0][0]                      
__________________________________________________________________________________________________
bn (BatchNormalization)         (None, 160, 160, 128 512         conv1[0][0]                      
__________________________________________________________________________________________________
act (Activation)                (None, 160, 160, 128 0           bn[0][0]                         
__________________________________________________________________________________________________
pool1 (Max

In [8]:
l = create_loss_function(anchor_width,
                         anchor_height,
                         LABEL_WEIGHT,
                         OFFSET_WEIGHT,
                         OFFSET_LOSS_WEIGHT,
                         EPSILON)

In [9]:
num_samples = get_num_samples(DATA_DIR)
print(f"Number of sampels: {num_samples}")
steps_epoch = num_samples // BATCHSIZE
if steps_epoch < 1:
    steps_epoch = 1
print(f"Steps per epoch: {steps_epoch}")

Number of sampels: 1000
Steps per epoch: 62


In [10]:
opt = optimizers.Adam(lr=1e-4, decay=1e-4)
model.compile(loss=l, optimizer=opt)

In [11]:
model.fit_generator(data_generator(DATA_DIR, BATCHSIZE, WIDTH, HEIGHT, anchor_width, anchor_height),
                    steps_per_epoch=steps_epoch,
                    epochs=1000,
                    verbose=1)

Epoch 1/1000
62/62 [==============================] - 40s 650ms/step - loss: 30.2188
Epoch 2/1000
62/62 [==============================] - 33s 535ms/step - loss: 17.9093
Epoch 3/1000
62/62 [==============================] - 33s 533ms/step - loss: 15.1765
Epoch 4/1000
62/62 [==============================] - 33s 534ms/step - loss: 12.4605
Epoch 5/1000
62/62 [==============================] - 33s 534ms/step - loss: 10.5809
Epoch 6/1000
62/62 [==============================] - 33s 533ms/step - loss: 9.5133
Epoch 7/1000
62/62 [==============================] - 33s 534ms/step - loss: 8.2038
Epoch 8/1000
62/62 [==============================] - 33s 533ms/step - loss: 7.9626
Epoch 9/1000
62/62 [==============================] - 33s 534ms/step - loss: 7.8649
Epoch 10/1000
62/62 [==============================] - 33s 534ms/step - loss: 7.2961
Epoch 11/1000
62/62 [==============================] - 33s 533ms/step - loss: 6.7516
Epoch 12/1000
62/62 [==============================] - 33s 534ms/step

KeyboardInterrupt: 